Import the data

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/inaturalist_12K"

Mounted at /content/gdrive
train  val


Install the dependencies

In [ ]:
!pip install wandb

Import the libraries

In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import matplotlib.pyplot as plt

In [10]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [11]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True)

Setup to wandb

In [6]:
import wandb
from wandb.keras import WandbCallback

In [7]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Preprocess the data

In [22]:
def dataProcess(batch_size = 32, data_aug = True, image_size = [299,299]):

    # data augmentation
    if(data_aug):
        train_datagen = ImageDataGenerator(
            rescale = 1./255,
            rotation_range = 90,
            shear_range = 0.2,
            zoom_range = 0.2,
            validation_split = 0.1,
            horizontal_flip = True)
    else:
        train_datagen = ImageDataGenerator( rescale = 1./255, validation_split = 0.1)

    test_datagen = ImageDataGenerator(rescale = 1./255)

    # generate train dataset
    train_generator = train_datagen.flow_from_directory(
        train_path,
        subset='training',
        target_size = image_size,
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True,
        seed = 45)

    # generate validation dataset
    val_generator = train_datagen.flow_from_directory(
        train_path,
        subset = 'validation',
        target_size = image_size,
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True,
        seed = 45)

    # generate test dataset
    test_generator = test_datagen.flow_from_directory(
        val_path,
        target_size = image_size,
        batch_size = batch_size,
        class_mode = 'categorical')

    '''
    class_names = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi',
               'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
    plt.figure(figsize=(10, 10))
    images, labels = val_generator.next()
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(class_names[np.where(labels[i] == 1)[0][0]])
        plt.axis("off")
    plt.show()
    '''
    return train_generator, val_generator, test_generator

Prepare the pre-trained model for our usage

In [24]:
def buildModel(pre_trained_model = 'InceptionV3',optimizer = 'adam', lr = 0.0001, image_size = [299,299], freeze = 0.9):
    print("image_size = ",image_size)
    # prepare the pre-trained model excluding the last layer
    if pre_trained_model == 'InceptionV3':
        pre_model = keras.applications.InceptionV3(input_shape = image_size + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'InceptionResNetV2':
        pre_model = keras.applications.InceptionResNetV2(input_shape = image_size + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'ResNet50':
        pre_model = keras.applications.ResNet50(input_shape = image_size + [3], weights = 'imagenet', include_top = False)
    if pre_trained_model == 'Xception':
        pre_model = keras.applications.Xception(input_shape = image_size + [3], weights = 'imagenet', include_top = False)
    
    '''
    # keep the trained weights of the pre trained model same
    for layer in pre_model.layers:
        layer.trainable = False
    '''

    # fine tuning the model
    k = len(pre_model.layers)
    upto = math.ceil(k*freeze)
    for i in range(upto):
        pre_model.layers[i] = False
    
    import os
    numberOfClasses = len(next(os.walk(train_path))[1])

    # flatten the last layer
    x = Flatten()(pre_model.output)
    # add output layer
    prediction = Dense(numberOfClasses, activation='softmax')(x)

    # create final model
    model = Model(inputs = pre_model.input, outputs = prediction)
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate = lr)
    if optimizer == 'adamax':
        opt = keras.optimizers.Adamax(learning_rate = lr)
    if optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate = lr)

    model.compile(optimizer = opt, 
                  loss = 'categorical_crossentropy', 
                  metrics = ['accuracy']
                  )
    return model

Sweep configurations

In [14]:
sweep_config = {
    'method' : 'random', 
    'metric' : {
        'name': 'val_accuracy',
        'goal': 'maximize'   
        },
    'parameters' : {
        'epochs': {'values' : [5,10]},
        'model' : {'values' : ['InceptionV3','InceptionResNetV2','ResNet50','Xception']},
        'optimizer' : {'values' : ['rmsprop','adam', 'adamax']},
        'batch_size' : {'values' : [16,32,64]},
        'lr' : {'values' : [1e-4,1e-5]},
        'data_aug' : {'values' : [True,False]},
        'freeze' : {'values' : [0.6,0.7,0.8,0.9,1]}
    }
}

Train the model

In [15]:
def train():
    config_defaults = {
        "epochs" : 5,
        "model" : 'InceptionV3',
        "optimizer" : 'adamax',
        "batch_size" : 32,
        "lr" : 0.0001,
        "data_aug" : True,
        "freeze" : 0.9
    }

    wandb.init(config = config_defaults,name = "CS6910-A2-P_A")
    config = wandb.config
    wandb.run.name = "epochs_{}_model_{}_opt_{}_bs_{}_lr_{}_da_{}".format(config.epochs,\
                                                                          config.model,\
                                                                          config.optimizer,\
                                                                          config.batch_size,\
                                                                          config.lr,\
                                                                          config.data_aug
                                                                          )
    wandb.run.save()

    # resize the images as per the pre trained model
    image_size = [299,299]
    if config.model == 'ResNet50' :
        image_size = [224,224]

    # process the data
    train_set, val_set, test_set = dataProcess(config.batch_size, config.data_aug, image_size)

    # build model 
    model = buildModel(config.model, config.optimizer, config.lr, image_size)

    # train the model
    trained_model = model.fit(train_set,
                              steps_per_epoch = train_set.samples // config.batch_size,
                              validation_data = val_set, 
                              validation_steps = val_set.samples // config.batch_size,
                              epochs = config.epochs,
                              callbacks=[WandbCallback(monitor='val_accuracy',mode='max')]
                            )

    # evaluate the model
    model.evaluate(test_set,
                   batch_size = config.batch_size,
                   callbacks=[WandbCallback()]
                  )

    wandb.finish()
    return model, trained_model

In [16]:
train_path = '/content/gdrive/My Drive/inaturalist_12K/train'
val_path = '/content/gdrive/My Drive/inaturalist_12K/val'


In [ ]:
#sweep_id = wandb.sweep(sweep_config,project = 'cs6910-As-2')

Create sweep with ID: piql1umf
Sweep URL: https://wandb.ai/pandp/cs6910-As-2/sweeps/piql1umf


In [ ]:
#wandb.agent(sweep_id, train, count=40)
wandb.agent('piql1umf', function = train, project = 'cs6910-As-2', count=40)

wandb: Agent Starting Run: plubogvc with config:
wandb: 	batch_size: 32
wandb: 	data_aug: False
wandb: 	epochs: 5
wandb: 	lr: 1e-05
wandb: 	model: InceptionV3
wandb: 	optimizer: rmsprop


Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
image_size =  [299, 299]
87924736/87910968 [==============================] - 2s 0us/step
Epoch 1/5
281/281 [==============================] - 1264s 4s/step - loss: 1.1362 - accuracy: 0.6551 - val_loss: 0.7497 - val_accuracy: 0.7692 - _timestamp: 1648841166.0000 - _runtime: 1273.0000
Epoch 2/5
281/281 [==============================] - 192s 684ms/step - loss: 0.6087 - accuracy: 0.8164 - val_loss: 0.6683 - val_accuracy: 0.8004 - _timestamp: 1648841382.0000 - _runtime: 1489.0000
Epoch 3/5
281/281 [==============================] - 190s 674ms/step - loss: 0.4612 - accuracy: 0.8580 - val_loss: 0.6408 - val_accuracy: 0.7994 - _timestamp: 1648841573.0000 - _runtime: 1680.0000
Epoch 4/5
281/281 [==============================] - 187s 665ms/step - loss: 0.3613 - accuracy: 0.8947 - val_loss: 0.6114 - val_accuracy: 0.8044 - _timestamp: 1648841771.0000 - _runtime: 1878.

accuracy,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▄▂▂▁
val_accuracy,▁▇▇██
val_loss,█▄▂▁▁
accuracy,0.92366
best_epoch,3
best_val_accuracy,0.80444
epoch,4
loss,0.27953
val_accuracy,0.80242


wandb: Agent Starting Run: 6tkx1zi0 with config:
wandb: 	batch_size: 16
wandb: 	data_aug: True
wandb: 	epochs: 5
wandb: 	lr: 1e-05
wandb: 	model: ResNet50
wandb: 	optimizer: adamax


Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
image_size =  [224, 224]
94781440/94765736 [==============================] - 1s 0us/step
Epoch 1/5
563/563 [==============================] - 297s 519ms/step - loss: 2.2962 - accuracy: 0.1273 - val_loss: 2.2765 - val_accuracy: 0.1583 - _timestamp: 1648842877.0000 - _runtime: 308.0000
Epoch 2/5
563/563 [==============================] - 282s 501ms/step - loss: 2.2750 - accuracy: 0.1395 - val_loss: 2.2640 - val_accuracy: 0.1552 - _timestamp: 1648843160.0000 - _runtime: 591.0000
Epoch 3/5
563/563 [==============================] - 281s 499ms/step - loss: 2.2605 - accuracy: 0.1569 - val_loss: 2.2612 - val_accuracy: 0.1663 - _timestamp: 1648843480.0000 - _runtime: 911.0000
Epoch 4/5
563/563 [==============================] - 281s 499ms/step - loss: 2.2536 - accuracy: 0.1567 - val_loss: 2.2432 - val_accuracy: 0.1603 - _timestamp: 1648843762.0000 - _runtime: 1193.0

...

wandb: Agent Starting Run: dj5il4mu with config:
wandb: 	batch_size: 32
wandb: 	data_aug: True
wandb: 	epochs: 5
wandb: 	lr: 1e-05
wandb: 	model: InceptionResNetV2
wandb: 	optimizer: rmsprop


Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
image_size =  [299, 299]
Epoch 1/5
281/281 [==============================] - 443s 2s/step - loss: 1.2573 - accuracy: 0.6174 - val_loss: 0.8936 - val_accuracy: 0.7278 - _timestamp: 1648844584.0000 - _runtime: 454.0000
Epoch 2/5
281/281 [==============================] - 419s 1s/step - loss: 0.8461 - accuracy: 0.7422 - val_loss: 0.8440 - val_accuracy: 0.7409 - _timestamp: 1648845004.0000 - _runtime: 874.0000
Epoch 3/5
281/281 [==============================] - 427s 2s/step - loss: 0.7686 - accuracy: 0.7647 - val_loss: 0.7882 - val_accuracy: 0.7571 - _timestamp: 1648845430.0000 - _runtime: 1300.0000
Epoch 4/5
281/281 [==============================] - 426s 2s/step - loss: 0.7228 - accuracy: 0.7746 - val_loss: 0.7635 - val_accuracy: 0.7692 - _timestamp: 1648845872.0000 - _runtime: 1742.0000
Epoch 5/5
63/63 [==============================] - 58s 903ms/step - loss

accuracy,▁▆▇██
epoch,▁▃▅▆█
loss,█▃▂▁▁
val_accuracy,▁▃▆█▆
val_loss,█▅▂▁▂
accuracy,0.78645
best_epoch,3
best_val_accuracy,0.76915
epoch,4
loss,0.68997
val_accuracy,0.75907


wandb: Agent Starting Run: k568x04q with config:
wandb: 	batch_size: 16
wandb: 	data_aug: True
wandb: 	epochs: 10
wandb: 	lr: 1e-05
wandb: 	model: Xception
wandb: 	optimizer: adamax


Found 9018 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
image_size =  [299, 299]
83697664/83683744 [==============================] - 1s 0us/step
Epoch 1/10
563/563 [==============================] - 419s 736ms/step - loss: 1.4179 - accuracy: 0.5873 - val_loss: 1.0480 - val_accuracy: 0.7167 - _timestamp: 1648846810.0000 - _runtime: 427.0000
Epoch 2/10
563/563 [==============================] - 429s 762ms/step - loss: 0.9539 - accuracy: 0.7181 - val_loss: 0.8872 - val_accuracy: 0.7359 - _timestamp: 1648847265.0000 - _runtime: 882.0000
Epoch 3/10
563/563 [==============================] - 424s 753ms/step - loss: 0.8480 - accuracy: 0.7387 - val_loss: 0.8236 - val_accuracy: 0.7329 - _timestamp: 1648847691.0000 - _runtime: 1308.0000
Epoch 4/10
157/563 [=======>......................] - ETA: 4:26 - loss: 0.8196 - accuracy: 0.7257